# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [5]:
# imports
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [6]:
# constants
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

In [13]:
# here is the question; type over this to ask something new

question = """
What is the difference between supervised and unsupervised learning in machine learning? Can you provide examples of each type of learning and explain their applications?
"""

In [20]:
def process_question(question):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers technical questions for the user."},
            {"role": "user", "content": question}
        ],
        max_tokens=1000,
        stream=True
    )
    result = ""
    for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            result += chunk.choices[0].delta.content
    return result.strip()

In [21]:
# Get gpt-4o-mini to answer, with streaming
def main():
    print(f"Question: {question.strip()}")
    answer = process_question(question)
    print(f"Answer: {answer}")
if __name__ == "__main__":
    main()

Question: What is the difference between supervised and unsupervised learning in machine learning? Can you provide examples of each type of learning and explain their applications?
Answer: In machine learning, supervised and unsupervised learning are two fundamental types of learning paradigms, each serving different purposes and applications. Here's an overview of each, along with examples and applications.

### Supervised Learning

**Definition:**  
Supervised learning involves training a model on a labeled dataset, which means that each training example is paired with an output label. The model learns to map input features to the corresponding output labels, enabling it to predict the label for new, unseen data.

**Examples:**
1. **Classification:** 
   - **Task:** Predicting whether an email is spam or not.
   - **Application:** Email filters that classify emails based on training data containing labeled examples of spam and non-spam emails.
   
2. **Regression:**
   - **Task:** Pr